Setup StreetFighter

In [ ]:
!pip install gym==0.21.0 gym-retro==0.7.1 importlib_metadata==4.13.0

In [14]:
# Import retro to play StreetFighter using a ROM
import retro
import time

In [ ]:
# List ganes
retro.data.list_games()

In [ ]:
# cd into roms folder and run - python -m retro.import .

In [ ]:
# Name should match the one in the list
# env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [ ]:
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [ ]:
env.observation_space

In [ ]:
env.action_space.sample()

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        #env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        print(reward)

In [55]:
env.close()

In [ ]:
info

Setup Environment

In [ ]:
!pip install opencv-python

In [ ]:
!pip install matplotlib

In [56]:
from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [57]:
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low = 0, high = 255, shape = (84, 84, 1), dtype = np.uint8)
        self.action_space = MultiBinary(12)
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions = retro.Actions.FILTERED)
    
    def reset(self):
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        self.score = 0
        return obs
    
    def preprocess(self, observation):
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (84, 84), interpolation = cv2.INTER_CUBIC)
        channels = np.reshape(resize, (84, 84, 1))
        return channels
    
    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs
        reward = info['score'] - self.score
        self.score = info['score']
        return frame_delta, reward, done, info

    def render(self, *args, **kwargs):
        self.game.render()
    
    def close(self):
        self.game.close()

In [ ]:
env = StreetFighter()

In [ ]:
env.observation_space.shape

In [ ]:
env.action_space.shape

In [ ]:
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        # env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        if reward > 0: 
            print(reward)

In [54]:
obs = env.reset()

In [ ]:
obs, reward, done, info = env.step(env.action_space.sample())

In [ ]:
plt.imshow(cv2.cvtColor(obs, cv2.COLOR_BGR2RGB))

Hyperparameter tune

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install "stable-baselines3[extra]==1.6.2" "optuna==2.10.1"

In [ ]:
!pip install torchvision==0.14.1 torchaudio==0.13.1

In [58]:
# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import os to deal with filepaths
import os

In [59]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [19]:
import torch

In [34]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0))

In [60]:
# Function to return test hyperparameters - define the object function
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [61]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = StreetFighter()
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=1000000)
        
        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        return -1000

In [62]:
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=10, n_jobs=1)

In [63]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_8_best_model.zip'))

Setup Callback

In [64]:
from stable_baselines3.common.callbacks import BaseCallback

In [65]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [66]:
CHECKPOINT_DIR = './train/'

In [67]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

Train Model

In [68]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
model_params = study.best_params
model_params['n_steps'] = 7424
model_params

In [75]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

In [76]:
model.load(os.path.join(OPT_DIR, 'trial_8_best_model.zip'))

In [77]:
model.learn(total_timesteps=7_000_000, callback=callback)

Evaluate the Model

In [78]:
model = PPO.load('./train/best_model_190000.zip')

In [ ]:
mean_reward, _ =evaluate_policy(model, env, render=True, n_eval_episodes=5)

In [80]:
mean_reward

Test Loop

In [81]:
obs = env.reset()

In [ ]:
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        #env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        if reward > 0: 
            print(reward)